<a href="https://colab.research.google.com/github/sanaz-mahmoudi/sanazmahmoudi/blob/main/Newcastle_University_Interview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pre Interview Task**

## Requirements

In [61]:
# PYOMO
!pip install -q pyomo
import pyomo.environ as pe

# NUMPY
import numpy as np

# GLPK
!apt-get install -y -qq glpk-utils
glpk = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')

# OTHER
import pandas as pd

## Input Data

In [100]:
print('==================================')
print('     Thermal Power Plant Data     ')
print('==================================')
data_gen = [[0,0,250,50]]
ng = len(data_gen)
gen = pd.DataFrame(data_gen,index=['g'+str(i) for i in range(ng)],columns=['bus','min','max','o_cost'])
print(gen)

print('==================================')
print('            Line Data             ')
print('==================================')
data_lin = [[0,1,100],[1,2,100],[0,2,100]]
nl = len(data_lin)
lin = pd.DataFrame(data_lin,index=['l'+str(i) for i in range(nl)],columns=['from','to','cap'])
print(lin)

print('==================================')
print('    Sulphur-Flow Battery Data     ')
print('==================================')
data_bat = [[1,0,100,24]]
nb = len(data_bat)
b = pd.DataFrame(data_bat,index=['b'+str(i) for i in range(nb)],columns=['bus','min','max','b_c'])
print(b)

print('==================================')
print('          Renewable Data          ')
print('==================================')
data_ren = [[1,0,150,0]]
nw = len(data_ren)
ren = pd.DataFrame(data_ren,index=['w'+str(i) for i in range(nw)],columns=['bus','min','max','o_cost'])
print(ren)

print('==================================')
print('         Load Demand Data         ')
print('==================================')
data_dem = [[2,100]]
nd = len(data_dem)
dem = pd.DataFrame(data_dem,index=['d'+str(i) for i in range(nd)],columns=['bus','max'])
print(dem)

print('==================================')
print(' Hourly Demand and Renewable Data ')
print('==================================')
h_data = pd.read_csv('https://raw.githubusercontent.com/sanaz-mahmoudi/sanazmahmoudi/refs/heads/main/Hourly%20Profiles.csv')

# Extract Days, Hours, Renewable Production, Consumption
h_data['day']  = h_data['Time'].str.split('T').str[0].str[-2:]
h_data['hour'] = h_data['Time'].str.split('T').str[1].str[:2]
h_data['prod'] = h_data['Normalised Renewable Production Profile']
h_data['cons'] = h_data['Normalised Electricity Consumption Profile']

# Create DataFrame for Hourly Load Demand
h_dem_b = pd.DataFrame(np.outer(h_data['cons'].values, dem['max'].values), columns=dem['bus'])   # Determine the Hourly Demand Profile of each Bus
h_dem_b['day']  = h_data['day'].values                                                           # Add Day from h_data
h_dem_b['hour'] = h_data['hour'].values                                                          # Add Hour from h_data
h_dem = h_dem_b.melt(id_vars=['day', 'hour'], var_name='bus', value_name='dem')                  # Melt into the Desired Format
print('Hourly Load Demand =\n', h_dem)

# Create DataFrame for Hourly Renewable Generation
h_ren_b = pd.DataFrame(np.outer(h_data['prod'].values, ren['max'].values), columns=ren['bus'])   # Determine the Hourly Renewable Generation Profile of each Bus
h_ren_b['day']  = h_data['day'].values                                                           # Add Day from h_data
h_ren_b['hour'] = h_data['hour'].values                                                          # Add Hour from h_data
h_ren = h_ren_b.melt(id_vars=['day', 'hour'], var_name='bus', value_name='gen')                  # Melt into the Desired Format
print('Hourly Renewable Generation =\n',h_ren)



     Thermal Power Plant Data     
    bus  min  max  o_cost
g0    0    0  250      50
            Line Data             
    from  to  cap
l0     0   1  100
l1     1   2  100
l2     0   2  100
    Sulphur-Flow Battery Data     
    bus  min  max  b_c
b0    1    0  100   24
          Renewable Data          
    bus  min  max  o_cost
w0    1    0  150       0
         Load Demand Data         
    bus  max
d0    2  100
 Hourly Demand and Renewable Data 
Hourly Load Demand =
     day hour bus        dem
0    01   00   2  19.199484
1    01   01   2  15.553450
2    01   02   2  10.650177
3    01   03   2   7.848147
4    01   04   2   7.711271
..   ..  ...  ..        ...
667  28   19   2  69.836309
668  28   20   2  62.527832
669  28   21   2  49.590679
670  28   22   2  34.204314
671  28   23   2  26.294096

[672 rows x 4 columns]
Hourly Renewable Generation =
     day hour bus        gen
0    01   00   1  24.037267
1    01   01   1  27.078261
2    01   02   1  34.188820
3    01   03   1 